# Experiment 003: Proper SA with Collision Constraints

This experiment implements simulated annealing with STRICT collision constraints:
1. Start from baseline (which has no overlaps)
2. Only accept moves that don't create overlaps
3. Use proper temperature schedule
4. Focus on small N values which contribute most to score

In [1]:
import numpy as np
import pandas as pd
import os
from numba import njit, prange
import math
import time
from shapely.geometry import Polygon
from shapely.strtree import STRtree
import shutil

os.chdir('/home/code/experiments/003_proper_sa')
print(f'Working directory: {os.getcwd()}')

Working directory: /home/code/experiments/003_proper_sa


In [2]:
# Tree polygon template
@njit
def make_polygon_template():
    tw=0.15; th=0.2; bw=0.7; mw=0.4; ow=0.25
    tip=0.8; t1=0.5; t2=0.25; base=0.0; tbot=-th
    x=np.array([0,ow/2,ow/4,mw/2,mw/4,bw/2,tw/2,tw/2,-tw/2,-tw/2,-bw/2,-mw/4,-mw/2,-ow/4,-ow/2],np.float64)
    y=np.array([tip,t1,t1,t2,t2,base,base,tbot,tbot,base,base,t2,t2,t1,t1],np.float64)
    return x, y

@njit
def get_bbox(xs, ys, degs, tx, ty):
    """Get bounding box side length for a configuration."""
    n = xs.size
    V = tx.size
    mnx = 1e300; mny = 1e300; mxx = -1e300; mxy = -1e300
    for i in range(n):
        r = degs[i] * math.pi / 180.0
        c = math.cos(r)
        s = math.sin(r)
        xi = xs[i]
        yi = ys[i]
        for j in range(V):
            X = c * tx[j] - s * ty[j] + xi
            Y = s * tx[j] + c * ty[j] + yi
            if X < mnx: mnx = X
            if X > mxx: mxx = X
            if Y < mny: mny = Y
            if Y > mxy: mxy = Y
    return max(mxx - mnx, mxy - mny)

@njit
def score_group(xs, ys, degs, tx, ty):
    n = xs.size
    side = get_bbox(xs, ys, degs, tx, ty)
    return side * side / n

tx, ty = make_polygon_template()
print('Template created')

Template created


In [3]:
# Load baseline submission
shutil.copy('/home/nonroot/snapshots/santa-2025/21105319338/code/datasets/santa-2025-csv/santa-2025.csv', 'submission.csv')

def strip(a):
    return np.array([float(str(v).replace('s', '')) for v in a], np.float64)

def load_submission(filepath):
    df = pd.read_csv(filepath)
    df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
    configs = {}
    for n, g in df.groupby('N'):
        xs = strip(g['x'].to_numpy())
        ys = strip(g['y'].to_numpy())
        ds = strip(g['deg'].to_numpy())
        configs[n] = {'x': xs, 'y': ys, 'deg': ds}
    return configs

configs = load_submission('submission.csv')

# Calculate initial score
initial_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        initial_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')

Initial score: 70.734327


In [4]:
# Overlap detection using Shapely
def get_shapely_polygon(cx, cy, deg, tx, ty):
    r = deg * np.pi / 180.0
    c = np.cos(r)
    s = np.sin(r)
    px = c * tx - s * ty + cx
    py = s * tx + c * ty + cy
    return Polygon(zip(px, py))

def has_overlap(xs, ys, degs, tx, ty):
    n = len(xs)
    if n <= 1:
        return False
    
    polygons = [get_shapely_polygon(xs[i], ys[i], degs[i], tx, ty) for i in range(n)]
    tree_index = STRtree(polygons)
    
    for i, poly in enumerate(polygons):
        indices = tree_index.query(poly)
        for idx in indices:
            if idx == i:
                continue
            if poly.intersects(polygons[idx]) and not poly.touches(polygons[idx]):
                return True
    return False

def has_overlap_single(xs, ys, degs, tx, ty, tree_idx):
    """Check if tree at tree_idx overlaps with any other tree."""
    n = len(xs)
    if n <= 1:
        return False
    
    poly_i = get_shapely_polygon(xs[tree_idx], ys[tree_idx], degs[tree_idx], tx, ty)
    
    for j in range(n):
        if j == tree_idx:
            continue
        poly_j = get_shapely_polygon(xs[j], ys[j], degs[j], tx, ty)
        if poly_i.intersects(poly_j) and not poly_i.touches(poly_j):
            return True
    return False

print('Overlap detection defined')

Overlap detection defined


In [ ]:
# Simulated Annealing with strict collision constraints
def sa_optimize(xs, ys, degs, tx, ty, n_iter=5000, T0=1.0, Tf=0.001, seed=42):
    """Simulated annealing with strict collision constraints."""
    np.random.seed(seed)
    n = len(xs)
    
    best_xs = xs.copy()
    best_ys = ys.copy()
    best_degs = degs.copy()
    best_side = get_bbox(best_xs, best_ys, best_degs, tx, ty)
    
    curr_xs = xs.copy()
    curr_ys = ys.copy()
    curr_degs = degs.copy()
    curr_side = best_side
    
    # Temperature schedule
    cooling = (Tf / T0) ** (1.0 / n_iter)
    T = T0
    
    # Move step sizes (decrease over time)
    step_sizes = [0.1, 0.05, 0.02, 0.01, 0.005, 0.002, 0.001]
    angle_steps = [10.0, 5.0, 2.0, 1.0, 0.5, 0.2, 0.1]
    
    accepted = 0
    improved = 0
    
    for it in range(n_iter):
        # Select step size based on progress
        step_idx = min(len(step_sizes) - 1, it * len(step_sizes) // n_iter)
        step = step_sizes[step_idx]
        angle_step = angle_steps[step_idx]
        
        # Pick random tree
        i = np.random.randint(n)
        
        # Save old position
        old_x, old_y, old_deg = curr_xs[i], curr_ys[i], curr_degs[i]
        
        # Random move type: 0=translate, 1=rotate, 2=both
        move_type = np.random.randint(3)
        
        if move_type == 0 or move_type == 2:
            # Translation in random direction
            dx = np.random.uniform(-step, step)
            dy = np.random.uniform(-step, step)
            curr_xs[i] += dx
            curr_ys[i] += dy
        
        if move_type == 1 or move_type == 2:
            # Rotation
            da = np.random.uniform(-angle_step, angle_step)
            curr_degs[i] = (curr_degs[i] + da) % 360
        
        # Check for overlap
        if has_overlap_single(curr_xs, curr_ys, curr_degs, tx, ty, i):
            # Revert - move creates overlap
            curr_xs[i], curr_ys[i], curr_degs[i] = old_x, old_y, old_deg
            continue
        
        # Calculate new side
        new_side = get_bbox(curr_xs, curr_ys, curr_degs, tx, ty)
        
        # Metropolis criterion
        delta = new_side - curr_side
        if delta < 0 or np.random.random() < np.exp(-delta / T):
            # Accept move
            curr_side = new_side
            accepted += 1
            
            if new_side < best_side:
                best_xs = curr_xs.copy()
                best_ys = curr_ys.copy()
                best_degs = curr_degs.copy()
                best_side = new_side
                improved += 1
        else:
            # Reject - revert
            curr_xs[i], curr_ys[i], curr_degs[i] = old_x, old_y, old_deg
        
        # Cool down
        T *= cooling
    
    return best_xs, best_ys, best_degs, best_side, accepted, improved

print('SA optimizer defined')

In [ ]:
# Test SA on a few small N values
print('Testing SA on small N values...')
start_time = time.time()

improved_configs = {}
for n in range(1, 31):  # Focus on N=1 to 30
    if n not in configs:
        continue
    
    c = configs[n]
    old_score = score_group(c['x'], c['y'], c['deg'], tx, ty)
    old_side = get_bbox(c['x'], c['y'], c['deg'], tx, ty)
    
    # Run SA with multiple restarts
    best_xs, best_ys, best_degs = c['x'].copy(), c['y'].copy(), c['deg'].copy()
    best_side = old_side
    
    for restart in range(5):
        new_xs, new_ys, new_degs, new_side, accepted, improved = sa_optimize(
            c['x'], c['y'], c['deg'], tx, ty,
            n_iter=3000, T0=0.5, Tf=0.0001, seed=42 + restart * 1000 + n
        )
        if new_side < best_side:
            best_xs, best_ys, best_degs = new_xs, new_ys, new_degs
            best_side = new_side
    
    new_score = best_side * best_side / n
    
    if new_score < old_score - 1e-10:
        print(f'N={n}: {old_score:.8f} -> {new_score:.8f} (improved by {old_score - new_score:.8f})')
        improved_configs[n] = {'x': best_xs, 'y': best_ys, 'deg': best_degs}
    else:
        if n <= 10:
            print(f'N={n}: {old_score:.8f} (no improvement)')

print(f'\nTime: {time.time() - start_time:.1f}s')
print(f'Improved {len(improved_configs)} configurations')

In [ ]:
# Update configs with improvements
for n, new_config in improved_configs.items():
    configs[n]['x'] = new_config['x']
    configs[n]['y'] = new_config['y']
    configs[n]['deg'] = new_config['deg']

# Calculate new total score
new_score = 0.0
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        new_score += score_group(c['x'], c['y'], c['deg'], tx, ty)

print(f'Initial score: {initial_score:.6f}')
print(f'New score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')

In [ ]:
# Validate for overlaps
print('Validating for overlaps...')
overlap_n = []
for n in range(1, 201):
    if n in configs:
        c = configs[n]
        if has_overlap(c['x'], c['y'], c['deg'], tx, ty):
            overlap_n.append(n)

if overlap_n:
    print(f'Overlaps found in N: {overlap_n}')
else:
    print('No overlaps detected')

In [ ]:
# Save submission
def save_submission(configs, filepath):
    rows = []
    for n in range(1, 201):
        if n in configs:
            c = configs[n]
            for i in range(len(c['x'])):
                rows.append({
                    'id': f'{n:03d}_{i}',
                    'x': f's{c["x"][i]}',
                    'y': f's{c["y"][i]}',
                    'deg': f's{c["deg"][i]}'
                })
    df = pd.DataFrame(rows)
    df.to_csv(filepath, index=False)
    print(f'Saved to {filepath}')

save_submission(configs, 'submission.csv')
shutil.copy('submission.csv', '/home/submission/submission.csv')
print('Submission saved to /home/submission/submission.csv')

In [ ]:
# Final summary
print(f'\n=== EXPERIMENT 003 SUMMARY ===')
print(f'Initial score: {initial_score:.6f}')
print(f'Final score: {new_score:.6f}')
print(f'Improvement: {initial_score - new_score:.6f}')
print(f'Overlaps: {len(overlap_n)}')